In [1]:
# Surprise 설치
!pip install scikit-surprise

In [4]:
import pandas as pd
import os
from surprise import Reader, Dataset

In [5]:
ratings=pd.read_csv("../data-files/recommendation/ratings1.csv")
ratings.to_csv("../data-files/recommendation/ratings_surprise.csv", index=False, header=False)
print(ratings.shape)
ratings.head(1)

(199999, 4)


,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044


In [59]:
# null 체크
ratings.isnull().sum()

userId       0
movieId      0
rating       0
timestamp    0
dtype: int64

In [6]:
# Reader라는 함수를 개별적으로 사용하여 csv파일의 포맷을 지정
from surprise import Reader
from surprise.model_selection import train_test_split

reader = Reader(line_format='user item rating timestamp', sep=',',
               rating_scale=(0.5, 5))
data = Dataset.load_from_file("../data-files/recommendation/ratings_surprise.csv", reader=reader)
train, test = train_test_split(data, test_size=0.2, random_state=42)

In [7]:
import pandas as pd
from surprise import Dataset, Reader
from surprise.model_selection import train_test_split

ratings = pd.read_csv("../data-files/recommendation/ratings1.csv")
reader = Reader(rating_scale=(0.5, 5))

# load_from_df사용해서 데이터프레임을 데이터셋으로 로드
# 인자에 userid-itemid-ratings 변수들이 포함된 데이터프레임형태로 넣어주면 됨!
data = Dataset.load_from_df(ratings[['userId','movieId','rating']],
                           reader=reader)
train, test = train_test_split(data, test_size=0.2, random_state=42)

In [62]:
# 모델 설정 및 학습
from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import train_test_split

# 내장 데이터인 무비렌즈 데이터 로드하고 학습/테스트 데이터로 분리
data = Dataset.load_builtin('ml-100k')
train, test = train_test_split(data, test_size=0.2, random_state=42)

# SVD 행렬 분해 알고리즘으로 SVD객체 생성 후 학습 수행
algo = SVD()
algo.fit(train)

In [63]:
# 예측 및 평가
prediction = algo.test(test)
print('prediction type: ', type(prediction),
     'size: ', len(prediction))
print()
print('prediction 결과값 5개 미리보기')
print(prediction[:5])

prediction type:  <class 'list'> size:  20000

prediction 결과값 5개 미리보기
[Prediction(uid='907', iid='143', r_ui=5.0, est=4.704240968243712, details={'was_impossible': False}), Prediction(uid='371', iid='210', r_ui=4.0, est=4.247999336347463, details={'was_impossible': False}), Prediction(uid='218', iid='42', r_ui=4.0, est=3.0333210139446694, details={'was_impossible': False}), Prediction(uid='829', iid='170', r_ui=4.0, est=3.830024545401087, details={'was_impossible': False}), Prediction(uid='733', iid='277', r_ui=1.0, est=2.8819945486378926, details={'was_impossible': False})]


In [64]:
# 원하는 값들을 튜플로 담아 추출
# user id, item id, 예측평점값들만 추출해서 하나의 튜플로 담겨있도록 하기
result = [(pred.uid, pred.iid, pred.est) for pred in prediction[:5]]
print(result)

[('907', '143', 4.704240968243712), ('371', '210', 4.247999336347463), ('218', '42', 3.0333210139446694), ('829', '170', 3.830024545401087), ('733', '277', 2.8819945486378926)]


In [65]:
# 개별 데이터에 대한 예측값 반환을 위해서 predict() 사용
# user id, item id는 문자열로 되어있기 때문에 문자열로 넣어주어야 함!
uid = str(196)
iid = str(302)
# 변수 순서 지켜주어서 넣어주어야 함!
pred = algo.predict(uid, iid)
print(pred)

user: 196        item: 302        r_ui = None   est = 4.19   {'was_impossible': False}


In [66]:
# RMSE 측정 (평균 제곱근 오차)
from surprise import SVD
from surprise import accuracy
from surprise import Dataset, Reader
from surprise.model_selection import train_test_split
import pandas as pd

ratings = pd.read_csv("./drive/MyDrive/data-files/ratings1.csv")
reader = Reader(rating_scale=(0.5, 5))

# load_from_df사용해서 데이터프레임을 데이터셋으로 로드
# 인자에 userid-itemid-ratings 변수들이 포함된 데이터프레임형태로 넣어주면 됨!
data = Dataset.load_from_df(ratings[['userId','movieId','rating']],
                           reader=reader)
train, test = train_test_split(data, test_size=0.2, random_state=42)

algo = SVD(n_factors=50, random_state=42)
algo.fit(train)
predictions = algo.test(test)
accuracy.rmse(predictions)

RMSE: 0.8687


0.8687000057890459

In [67]:
from surprise.model_selection import cross_validate

# Pandas DF 형태로 데이터 로드
ratings = pd.read_csv("./drive/MyDrive/data-files/ratings1.csv")
reader = Reader(rating_scale=(0.5, 5))

data = Dataset.load_from_df(ratings[['userId','movieId','rating']],
                           reader=reader)

algo = SVD(n_factors=50, random_state=42)
# cross_validate에는 파라미터를 입력시켜 놓은 모델을 인자로 넣어주자!
cross_validate(algo, data, measures=['RMSE','MAE'], cv=5,
              verbose=True)
# MAE (평균 제곱 오차) 

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8636  0.8616  0.8660  0.8690  0.8679  0.8656  0.0027  
MAE (testset)     0.6629  0.6613  0.6656  0.6662  0.6632  0.6638  0.0018  
Fit time          5.99    6.01    6.04    6.01    6.01    6.01    0.02    
Test time         0.31    0.32    0.31    0.48    0.51    0.39    0.09    


{'fit_time': (5.990783929824829,
  6.009910821914673,
  6.0421507358551025,
  6.007091760635376,
  6.011822700500488),
 'test_mae': array([0.66288084, 0.66130473, 0.6655826 , 0.6662153 , 0.66315168]),
 'test_rmse': array([0.86355006, 0.86157172, 0.86595877, 0.86903038, 0.86788295]),
 'test_time': (0.30698132514953613,
  0.3242759704589844,
  0.30614662170410156,
  0.48218655586242676,
  0.5073792934417725)}

In [68]:
# 그리드를 사용한 복수 하이퍼 파라미터 최적화
from surprise.model_selection import GridSearchCV

# GridSearch 할 파라미터 사전적으로 정의
param_grid = {'n_epochs':[20,40], 'n_factors':[50, 100,200]}

# GridSearchCV는 cross_validate와는 달리 인자에 알고리즘 자체를 넣어준다!
grid = GridSearchCV(SVD, param_grid=param_grid,
                   measures=['rmse','mae'], cv=3) # measure을 소문자로 해줘야함!
# GridSearchCV로 데이터 학습시키기
grid.fit(data)

# 최고의 score와 그 때의 파라미터 출력
print(grid.best_score['rmse'])
print(grid.best_params['rmse'])

0.8704513062794211
{'n_epochs': 20, 'n_factors': 50}


In [71]:
# 추천시스템 구현하기
from surprise.dataset import DatasetAutoFolds
reader = Reader(line_format='user item rating timestamp', sep=',', rating_scale=(0.5, 5))
data_folds = Dataset.load_from_file("../data-files/recommendation/ratings_surprise.csv', reader=reader)

#전체 데이터를 학습데이터로 생성함. 
trainset = data_folds.build_full_trainset()

algo = SVD(n_epochs=20, n_factors=50, random_state=0) 
algo.fit(trainset)

# 영화에 대한 상세 속성 정보 DataFrame로딩 
movies = pd.read_csv("../data-files/recommendation/moives.csv')

# userId=9 의 movieId 데이터 추출하여 movieId=44 데이터가 있는지 확인. 
movieIds = ratings[ratings['userId']==10]['movieId']
if movieIds[movieIds==44].count() == 0: 
    print('사용자 아이디 9는 영화 아이디 44의 평점 없음')

print(movies[movies['movieId']==44])


사용자 아이디 9는 영화 아이디 44의 평점 없음
    movieId                 title                    genres
43       44  Mortal Kombat (1995)  Action|Adventure|Fantasy


In [73]:
# 추천예측 평점확인
uid = str(9)
iid = str(44)
pred = algo.predict(uid, iid, verbose=True)

user: 9          item: 44         r_ui = None   est = 3.27   {'was_impossible': False}


In [74]:
from surprise.dataset import DatasetAutoFolds
from surprise.dataset import Reader
from surprise import SVD

reader = Reader(line_format='user item rating timestamp', sep=',',
               rating_scale=(0.5, 5))

# DatasetAutoFolds 클래스를 사용해서 개별적으로 생성
# index와 header가 없는 상태로 재생성했던 ratings_surprise.csv파일에 기반
data_folds = Dataset.load_from_file('./drive/MyDrive/data-files/ratings_surprise.csv', reader=reader)

# 위에서 개별적으로 생성한 csv파일을 학습데이터로 생성
trainset = data_folds.build_full_trainset()
algo = SVD(n_factors=50, n_epochs=20, random_state=42)
algo.fit(trainset)

# 영화에 대한 정보 데이터 로딩
movise = pd.read_csv("./drive/MyDrive/data-files/moives.csv")
ratings = pd.read_csv("./drive/MyDrive/data-files/ratings1.csv")
# 특정 사용자 9번의 movieId를 추출해서 특정 영화에 대한 평점 있는지 확인
movieIds = ratings[ratings['userId']==9]['movieId']
if movieIds[movieIds==44].count() == 0:
    print('user id=10인 사람은 movie id=44에 대한 평점이 없음')
    
# 영화에 대한 정보 데이터에서 movieId가 44인 영화가 무엇인지 출력
print(movise[movise['movieId']==44])

def get_unseen_surprise(ratings, movies, userId):
    # 특정 유저가 본 movie id들을 리스트로 할당
    seen_movies = ratings[ratings['userId']==userId]['movieId'].tolist()
    # 모든 영화들의 movie id들 리스트로 할당
    total_movies = movies['movieId'].tolist()
    
    # 모든 영화들의 movie id들 중 특정 유저가 본 movie id를 제외한 나머지 추출
    unseen_movies = [movie for movie in total_movies if movie not in seen_movies]
    print(f'특정 {userId}번 유저가 본 영화 수: {len(seen_movies)}\n추천한 영화 개수: {len(unseen_movies)}\n전체 영화수: {len(total_movies)}')
    
    return unseen_movies

    

user id=10인 사람은 movie id=44에 대한 평점이 없음
    movieId                 title                    genres
43       44  Mortal Kombat (1995)  Action|Adventure|Fantasy


In [75]:

def recomm_movie_by_surprise(algo, userId, unseen_movies, top_n=10):
    # 알고리즘 객체의 predict()를 이용해 특정 userId의 평점이 없는 영화들에 대해 평점 예측
    predictions = [algo.predict(str(userId), str(movieId)) for movieId in unseen_movies]
    
    # predictions는 Prediction()으로 하나의 객체로 되어있기 때문에 예측평점(est값)을 기준으로 정렬해야함
    # est값을 반환하는 함수부터 정의. 이것을 이용해 리스트를 정렬하는 sort()인자의 key값에 넣어주자!
    def sortkey_est(pred):
        return pred.est
    
    # sortkey_est함수로 리스트를 정렬하는 sort함수의 key인자에 넣어주자
    # 리스트 sort는 디폴트값이 inplace=True인 것처럼 정렬되어 나온다. reverse=True가 내림차순
    predictions.sort(key=sortkey_est, reverse=True)
    # 상위 n개의 예측값들만 할당
    top_predictions = predictions[:top_n]
    
    # top_predictions에서 movie id, rating, movie title 각 뽑아내기
    top_movie_ids = [int(pred.iid) for pred in top_predictions]
    top_movie_ratings = [pred.est for pred in top_predictions]
    top_movie_titles = movies[movies.movieId.isin(top_movie_ids)]['title']
    # 위 3가지를 튜플로 담기
    # zip함수를 사용해서 각 자료구조(여기선 리스트)의 똑같은 위치에있는 값들을 mapping
    # zip함수는 참고로 여러개의 문자열의 똑같은 위치들끼리 mapping도 가능!
    top_movie_preds = [(ids, rating, title) for ids, rating, title in zip(top_movie_ids, top_movie_ratings, top_movie_titles)]
    
    return top_movie_preds

### 위에서 정의한 함수를 사용해 특정 유저의 추천 영화들 출력해보기
unseen_lst = get_unseen_surprise(ratings, movies, 9)
top_movies_preds = recomm_movie_by_surprise(algo, 9, unseen_lst,
                                           top_n=10)
print()
print('#'*8,'Top-10 추천영화 리스트','#'*8)

# top_movies_preds가 여러가지의 튜플을 담고 있는 리스트이기 때문에 반복문 수행
for top_movie in top_movies_preds:
    print('* 추천 영화 이름: ', top_movie[2])
    print('* 해당 영화의 예측평점: ', top_movie[1])
    print() 

특정 9번 유저가 본 영화 수: 178
추천한 영화 개수: 62245
전체 영화수: 62423

######## Top-10 추천영화 리스트 ########
* 추천 영화 이름:  Braveheart (1995)
* 해당 영화의 예측평점:  4.962341530052258

* 추천 영화 이름:  When We Were Kings (1996)
* 해당 영화의 예측평점:  4.858139155559446

* 추천 영화 이름:  Double Life of Veronique, The (Double Vie de Véronique, La) (1991)
* 해당 영화의 예측평점:  4.75893410505591

* 추천 영화 이름:  Quiet Man, The (1952)
* 해당 영화의 예측평점:  4.718414977554937

* 추천 영화 이름:  Gandhi (1982)
* 해당 영화의 예측평점:  4.708577614008236

* 추천 영화 이름:  Good Will Hunting (1997)
* 해당 영화의 예측평점:  4.7027054227400775

* 추천 영화 이름:  King and I, The (1956)
* 해당 영화의 예측평점:  4.698636320670707

* 추천 영화 이름:  Gladiator (1992)
* 해당 영화의 예측평점:  4.694964253797825

* 추천 영화 이름:  Inception (2010)
* 해당 영화의 예측평점:  4.694016028669485

* 추천 영화 이름:  Planet Earth II (2016)
* 해당 영화의 예측평점:  4.691950123280238

